In [1]:
import pandas as pd
import numpy as np
import sklearn.manifold as sk_m
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import NearestNeighbors

In [2]:
df = pd.read_csv("beer_reviews.csv")

In [3]:
beer_abv_null = list(df["beer_beerid"].loc[df["beer_abv"].isna() == True].index)
df.drop(index = beer_abv_null, inplace=True)

In [4]:
df_new=df[["review_aroma", 
           "review_appearance",
           "beer_style",
           "review_palate",
           "review_taste",
           "review_profilename"]]

In [5]:
beer_features=df[["beer_beerid",
                  "brewery_name",
                  "beer_name",
                  "beer_style",
                  "review_overall",
                  "beer_abv",
                  "review_profilename"]].groupby("beer_beerid").agg({"brewery_name":"first",
                                                                    "beer_name":"first",
                                                                    "beer_style":"first",
                                                                    "review_overall":"mean",
                                                                    "beer_abv":"mean",
                                                                    "review_profilename":"count"}).reset_index()

In [6]:
beer_style_features=df_new.groupby('beer_style').mean()

In [7]:
model_knn = NearestNeighbors(metric="euclidean",
                             algorithm="brute",
                             n_jobs=-1)

model_knn.fit(beer_style_features)

NearestNeighbors(algorithm='brute', metric='euclidean', n_jobs=-1)

In [8]:
def user_feature(user_features):
    user_features_df= pd.DataFrame(user_features, index= [0])
    return user_features_df 

In [9]:
def model(user_feat):
    prediction_knn = model_knn.kneighbors(user_feat, n_neighbors=3)
    return prediction_knn

In [10]:
def dis_recommendetions(prediction_knn):
    print('Based on your preferences we recommend:')
    print('')
    for i, beer_style in enumerate(prediction_knn[1][0]):
        style=beer_style_features.iloc[beer_style].name
        beer_style_df=beer_features.loc[beer_features["beer_style"]==style].sort_values("review_profilename",ascending=False).head(3)
        print('')
        print(f"-{i+1}-\nBeer Style: {style}, we suggest the 3 most popular beers:")
        print('----------------------------------------------------------------')
        for index in beer_style_df.index:
            print('==> {}:\n    - Alcohol:   {}%\n    - Brewery:   {}\n    - Reviews:   {}\n    - Avg Score: {}'.format(beer_style_df.loc[index]["beer_name"],round(beer_style_df.loc[index]["beer_abv"],1),beer_style_df.loc[index]["brewery_name"],beer_style_df.loc[index]["review_profilename"],round(beer_style_df.loc[index]["review_overall"],1)))
            print('')

In [11]:
def beer_rec(user_features):
    user_feat = user_feature(user_features)
    knn_model = model(user_feat)
    dis_recommendetions(knn_model)
    

In [12]:
beer_rec(user_features= {"review_aroma" : 5,
                         "review_appearance" :2 ,
                         "review_palate": 3,
                         "review_taste": 1})

Based on your preferences we recommend:


-1-
Beer Style: Euro Strong Lager, we suggest the 3 most popular beers:
----------------------------------------------------------------
==> Baltika #9 Extra (Strong):
    - Alcohol:   8.0%
    - Brewery:   Baltika Breweries
    - Reviews:   344
    - Avg Score: 2.8

==> Elephant Beer:
    - Alcohol:   7.2%
    - Brewery:   Carlsberg Danmark A/S
    - Reviews:   270
    - Avg Score: 2.8

==> Lion Imperial Premium Malt Pilsner:
    - Alcohol:   8.8%
    - Brewery:   Ceylon / Lion Brewery Limited
    - Reviews:   113
    - Avg Score: 2.9


-2-
Beer Style: Happoshu, we suggest the 3 most popular beers:
----------------------------------------------------------------
==> Sparkling Hop:
    - Alcohol:   5.0%
    - Brewery:   Kirin Brewery Company, Limited
    - Reviews:   34
    - Avg Score: 3.3

==> Kirin Tanrei:
    - Alcohol:   5.0%
    - Brewery:   Kirin Brewery Company, Limited
    - Reviews:   33
    - Avg Score: 3.0

==> Asahi Honnama Red:
  